In [ ]:
import sys
sys.path.append(r'..\src')

import pandas as pd

import pylab as plt

import seaborn as sns

import numpy as np

from src import getGraphic
from ufc_fights import set_names_upper, set_name_winner, fix_columns, set_nan_columns, add_fighterid
from ufc_fighters import load_ufc, set_fighter_upper, clean_fighters

In [ ]:
fighter = pd.read_csv(r'..\data\raw_fighter_details.csv')

data = pd.read_csv(r'..\data\data.csv')

ufc_fighters = pd.read_csv(r'..\data\ufc_fighters_official.csv')

# TABLA LUCHADORES

In [ ]:
set_fighter_upper(fighter)

fighters = clean_fighters(fighter, ufc_fighters)

fighters

# TABLA FIGHTS

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# Las reglas oficiales de la UFC no entraron hasta el año 2001, así que solo cogemos combates por encima del año 2002
data = data[data.date>'2002-01-01']
# B_draw y R_draw son columnas con todos los valores 0
data = data.drop(['B_draw','R_draw'], axis=1)

In [ ]:
set_names_upper(data)

data.head()

In [ ]:
'''set_name_winner(data)

data.head()'''

In [ ]:
fix_columns(data)

data

In [ ]:
set_nan_columns(data)

data

In [ ]:
data.select_dtypes(include=['object','bool'])

In [ ]:
data.info()

In [ ]:
# ¿Borrar LOCATION?


In [ ]:
'''# buscando colinealidad

plt.figure(figsize=(15, 10))

sns.set(style='white')

mask=np.triu(np.ones_like(data1.iloc[:,31:55].corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)


sns.heatmap(data1.iloc[:,31:55].corr(),
           mask=mask,
          cmap=cmap,
          center=0,
          square=True,
          linewidths=0.5,
          cbar_kws={'shrink': 0.5},
           annot=True);'''